<a href="https://colab.research.google.com/github/ImpulsoGov/mensageria-mvp/blob/main/passo_2_match_com_equipe_e_upload_turn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Match com equipes da base de dados da Turn e upload de contatos

#### Configurações iniciais do ambiente

In [3]:
!pip install python-dotenv

In [4]:
import pandas as pd
import io
import numpy as np
from sklearn.model_selection import train_test_split
from google.oauth2 import service_account
from datetime import datetime
from google.cloud import bigquery
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/credenciais_service_account.json')
project_id = 'predictive-keep-314223'
client = bigquery.Client(credentials= credentials,project=project_id)
import requests
import json
import time
from dotenv import load_dotenv
import os

#### Consulta e preparação dos dados

In [7]:
# carregando a tabela de usuários selecionados
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens`
    WHERE mvp_data_envio = current_date()
"""

query_job = client.query(query)
df_historico_envio_mensagens = query_job.to_dataframe()

In [8]:
# carregando dados dos municipios da tabela do ibge
query = """
    SELECT *
    FROM `predictive-keep-314223.lista_de_codigos.municipios_ibge`
"""

query_job = client.query(query)
df_ibge = query_job.to_dataframe()

In [9]:
df_ibge['id_sus'] = df_ibge['id_sus'].astype(str)
df_historico_envio_mensagens['municipio_id_sus'] = df_historico_envio_mensagens['municipio_id_sus'].astype(str)

In [10]:
# merge da tabela de usuários com de municípios
df_historico_envio_mensagens = pd.merge(df_historico_envio_mensagens, df_ibge[['id_sus','nome','uf_sigla']], left_on=['municipio_id_sus'],right_on=['id_sus'] , how='left')

In [11]:
# carregando a view com dados da Turn io
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.contact_details_turnio`
"""

query_job = client.query(query)
df_contatos_turnio = query_job.to_dataframe()

In [12]:
df_contatos_turnio[['municipio', 'uf']] = df_contatos_turnio['details_municipio'].str.split(' - ', expand=True)

In [14]:
# merge das tabelas de dados dos usuários e estabelecimento
df = pd.merge(df_historico_envio_mensagens, df_contatos_turnio, left_on=['equipe_nome','municipio','uf_sigla'], right_on=['details_equipe_nome','municipio','uf'], how='left')

In [26]:
df_envio_turn = df[['municipio',
                    'uf_sigla',
                    'municipio_id_sus',
                    'equipe_ine',
                    'equipe_nome',
                    'linha_cuidado',
                    'nome_do_paciente',
                    'data_de_nascimento',
                    'celular_tratado',
                    'mvp_tipo_grupo',
                    'mvp_data_envio',
                    'mvp_grupo',
                    'details_horarios_cronicos',
                    'details_telefone',
                    'details_estabelecimento_endereco',
                    'details_estabelecimento_telefone',
                    'details_estabelecimento_nome',
                    'details_horarios_cito',
                    'details_estabelecimento_documentos',
                    'details_estabelecimento_horario'
                    ]]

In [27]:
df_envio_turn = df_envio_turn.rename(columns={'details_horarios_cronicos':'horarios_cronicos',
                                              'details_telefone':'telefone',
                                              'details_estabelecimento_endereco':'estabelecimento_endereco',
                                              'details_estabelecimento_telefone':'estabelecimento_telefone',
                                              'details_estabelecimento_nome':'estabelecimento_nome',
                                              'details_horarios_cito':'horarios_cito',
                                              'details_estabelecimento_documentos':'estabelecimento_documentos',
                                              'details_estabelecimento_horario':'estabelecimento_horario'})

#### Configuração do envio para a Turn io

In [22]:
load_dotenv('/content/drive/MyDrive/env.env')
tokens_municipios = [
    {"municipio": "Paulo Ramos", "id_sus": "210810", "token": os.getenv('ENV_PAULORAMOS_MA')},
    {"municipio": "Pacoti", "id_sus": "230980", "token": os.getenv('ENV_PACOTI_CE')},
    {"municipio": "Marajá do Sena", "id_sus": "210635", "token": os.getenv('ENV_MARAJADOSENA_MA')},
    {"municipio": "Monsenhor Tabosa", "id_sus": "230860", "token": os.getenv('ENV_MONSENHORTABOSA_CE')},
    {"municipio": "Lago Verde", "id_sus": "210590", "token": os.getenv('ENV_LAGOVERDE_MA')},
    {"municipio": "Alagoinha", "id_sus": "260060", "token": os.getenv('ENV_ALAGOINHA_PE')},
    {"municipio": "Baraúna", "id_sus": "240145", "token": os.getenv('ENV_BARAUNA_RN')},
    {"municipio": "Jucuruçu", "id_sus": "291845", "token": os.getenv('ENV_JUCURUCU_BA')},
    {"municipio": "Vitorino Freire", "id_sus": "211300", "token": os.getenv('ENV_VITORINOFREIRE_MA')},

]

In [25]:
def get_token(municipio_id_sus):
    for token_info in tokens_municipios:
        if token_info['id_sus'] == municipio_id_sus:
            return token_info['token']
    return None

def send_data(df, municipio_id_sus):
    token = get_token(municipio_id_sus)
    if not token:
        print(f"Token não encontrado para {municipio_id_sus}")
        return

    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/vnd.v1+json',
        'Content-Type': 'application/json'
    }

    df_filtered = df[df['municipio_id_sus'] == municipio_id_sus]

    for i, row in df_filtered.iterrows():
        data_message = {
            "preview_url": False,
            "recipient_type": "individual",
            "to": str(row.celular_tratado),
            "type": "text",
            "text": {"body": "Este número pertence a ImpulsoGov."}
        }
        url_message = 'https://whatsapp.turn.io/v1/messages'

        try:
            response_message = requests.post(url_message, headers=headers, json=data_message)
            print(f"Resposta da mensagem para {row.celular_tratado}: {response_message.text}")
        except Exception as e:
            print(f"Erro ao enviar mensagem para {row.celular_tratado}: {e}")
            continue

        time.sleep(1)

        #atualiza opted_in
        json_data_profile = {
            "opted_in": True,
        }
        url_profile = f'https://whatsapp.turn.io/v1/contacts/{row.celular_tratado}/profile'

        try:
            response_profile = requests.patch(url_profile, headers=headers, json=json_data_profile)
            print(f"Resposta do perfil para {row.celular_tratado}: {response_profile.text}")
        except Exception as e:
            print(f"Erro ao atualizar perfil de {row.celular_tratado}: {e}")

        time.sleep(1)

for municipio_id_sus in df_envio_turn['municipio_id_sus'].unique():
    send_data(df_envio_turn, municipio_id_sus)


Resposta da mensagem para 5511952542220: {"meta":{"version":"4.684.1","backend":{"name":"WhatsApp","version":"latest"},"api_status":"stable"},"errors":[{"code":133010,"error_subcode":2593006,"error_user_msg":"Account does not exist in Cloud API, please use /register API to create an account first.","error_user_title":"Account Not Exist","fbtrace_id":"A6IszZmCNg-7Y07TfN0Vda2","is_transient":false,"message":"The account is not registered","type":"OAuthException"}]}
Resposta do perfil para 5511952542220: {"fields":{"birthday":null,"equipe_ine":null,"equipe_nome":null,"estabelecimento_documentos":null,"estabelecimento_endereco":null,"estabelecimento_horario":null,"estabelecimento_nome":null,"estabelecimento_telefone":null,"first_message_received_at":null,"horarios_cito":null,"horarios_cronicos":null,"is_blocked":false,"language":null,"last_message_received_at":null,"last_message_sent_at":null,"last_seen_at":null,"linha_de_cuidado":null,"location":null,"municipio":null,"municipio_id_sus":nu